In [68]:
import os
import json
import matplotlib.pyplot as plt
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist, cifar10, cifar100
import tensorflow.keras.backend as K
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [69]:
# Define folder structure
BASE_DIR = "saved_models"
PLOT_DIR = "training_graphs"
RESULTS_DIR = "results"

os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

In [70]:
# Custom activation functions
def penalized_tanh(x, alpha=0.25):  # alpha is a hyperparameter (default 0.25)
    return tf.where(x >= 0, tf.tanh(x), alpha * tf.tanh(x))

def eliSH(x):
    return x * tf.nn.sigmoid(x) * (1 + tf.nn.tanh(x))

def mish(x):
    return x * tf.nn.tanh(K.softplus(x))  # softplus(x) = log(1 + exp(x))

def rsigelu(x):
    return x * tf.nn.sigmoid(x) + tf.nn.elu(x)

def tanh_exp(x):
    return x * tf.tanh(K.exp(x))

def pflu(x, alpha=0.1, beta=1.0):
    return tf.where(x >= 0, x + beta, alpha * (tf.exp(x) - 1))

def gcu(x):
    return x * tf.cos(x)

def hcLSH(x):
    return x * tf.nn.sigmoid(x) * K.log(K.sigmoid(x) + K.epsilon())


In [71]:
activation_functions = {
    "relu": tf.nn.relu,
    "tanh": tf.nn.tanh,
    "sigmoid": tf.nn.sigmoid,
    "hcLSH": hcLSH,
    "penalized_tanh": penalized_tanh,
    "eliSH": eliSH,
    "mish": mish,
    "rsigelu": rsigelu,
    "tanh_exp": tanh_exp,
    "pflu": pflu,
    "gcu": gcu
}

In [ ]:
activation_colors = {
    "relu": "blue",
    "tanh": "green",
    "sigmoid": "red",
    "hcLSH": "purple",
    "penalized_tanh": "orange",
    "eliSH": "cyan",
    "mish": "magenta",
    "rsigelu": "brown",
    "tanh_exp": "pink",
    "pflu": "gray",
    "gcu": "olive"
}

In [72]:
def load_svhn():
    train_data = loadmat("path/to/train_32x32.mat")
    test_data = loadmat("path/to/test_32x32.mat")

    X_train = np.moveaxis(train_data["X"], -1, 0)  # Move axis to match TF format
    y_train = train_data["y"].flatten() % 10  # Normalize labels (SVHN uses 1-10)
    
    X_test = np.moveaxis(test_data["X"], -1, 0)
    y_test = test_data["y"].flatten() % 10

    return X_train, X_test, y_train, y_test


In [73]:
def load_dataset(dataset_name):
    if dataset_name == "fashion_mnist":
        (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
        X_train = X_train[..., np.newaxis]  # Add channel dimension
        X_test = X_test[..., np.newaxis]

    elif dataset_name == "cifar10":
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    elif dataset_name == "cifar100":
        (X_train, y_train), (X_test, y_test) = cifar100.load_data()

    elif dataset_name == "svhn":
        X_train, X_test, y_train, y_test = load_svhn()

    else:
        raise ValueError("Dataset not supported!")

    # Normalize data
    X_train, X_test = X_train / 255.0, X_test / 255.0

    return X_train, X_test, y_train, y_test

In [74]:
def create_cnn_model(input_shape, activation_name, num_classes):
    activation_func = activation_functions[activation_name]

    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=input_shape),
        Lambda(activation_func),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=64, kernel_size=(3, 3), padding="same"),
        Lambda(activation_func),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256),
        Lambda(activation_func),
        Dropout(0.1),
        Dense(128),
        Lambda(activation_func),
        Dropout(0.1),
        Dense(num_classes, activation="softmax")  # Multi-class classification
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [75]:
def create_rnn_model(activation_name, input_shape, num_classes):
    activation_fn = activation_functions[activation_name]  # Default to Tanh if not found

    model = tf.keras.Sequential([
        tf.keras.layers.SimpleRNN(64, activation=activation_fn, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.SimpleRNN(128, activation=activation_fn, return_sequences=False),
        tf.keras.layers.Dense(64, activation=activation_fn),
        tf.keras.layers.Dense(128, activation=activation_fn),
        tf.keras.layers.Dense(num_classes, activation="softmax")  # Output layer
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


In [76]:
# Function to save model properly
def save_model(model, dataset_name, activation_name):
    model_dir = os.path.join("saved_models", dataset_name)
    os.makedirs(model_dir, exist_ok=True)  # Ensure directory exists

    # Save model weights (only weights, NOT full model)
    model_path = os.path.join(model_dir, f"cnn_{dataset_name}_{activation_name}.h5")
    model.save_weights(model_path)  # Save weights
    print(f"✅ Model weights saved: {model_path}")

    # Save metadata separately (since model structure cannot be serialized)
    metadata = {
        "dataset": dataset_name,
        "activation": activation_name,
    }
    metadata_path = os.path.join(model_dir, f"metadata_{dataset_name}_{activation_name}.json")
    with open(metadata_path, "w") as json_file:
        json.dump(metadata, json_file, indent=4)

    print(f"✅ Metadata saved: {metadata_path}")

In [77]:
# Function to reload the model
def load_saved_model(dataset_name, activation_name, input_shape, num_classes):
    model_dir = os.path.join("saved_models", dataset_name)

    # Rebuild the model
    model = create_cnn_model(input_shape, activation_name, num_classes)

    # Load saved weights
    model_path = os.path.join(model_dir, f"cnn_{dataset_name}_{activation_name}.h5")
    model.load_weights(model_path)
    print(f"✅ Model weights loaded: {model_path}")

    return model

In [78]:
# Function to plot training results
def plot_training(history, dataset_name, activation_name):
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    # Accuracy Plot
    ax[0].plot(history.history["accuracy"], label="Train Accuracy")
    ax[0].plot(history.history["val_accuracy"], label="Validation Accuracy")
    ax[0].set_title(f"Accuracy ({dataset_name}, {activation_name})")
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")
    ax[0].legend()

    # Loss Plot
    ax[1].plot(history.history["loss"], label="Train Loss")
    ax[1].plot(history.history["val_loss"], label="Validation Loss")
    ax[1].set_title(f"Loss ({dataset_name}, {activation_name})")
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    ax[1].legend()

    # Save plot
    plot_filename = f"{PLOT_DIR}/training_{dataset_name}_{activation_name}.png"
    plt.savefig(plot_filename)
    plt.close()
    print(f"📊 Plot saved as {plot_filename}")

In [79]:
# Function to save test results per dataset
def save_test_results(dataset_name, activation_name, test_loss, test_acc):
    results_file = os.path.join(RESULTS_DIR, f"{dataset_name}_results.csv")
    
    file_exists = os.path.isfile(results_file)

    with open(results_file, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Activation Function", "Test Loss", "Test Accuracy"])  # Header
        writer.writerow([activation_name, test_loss, test_acc])  # Write results
    
    print(f"✅ Test accuracy saved in {results_file}")

In [80]:
datasets = ["fashion_mnist", "cifar10", "cifar100"]  # Change to "cifar10", "cifar100", "svhn"
activation_functions = {
    "relu": tf.nn.relu,
    "tanh": tf.nn.tanh,
    "sigmoid": tf.nn.sigmoid,
    "hcLSH": hcLSH,
    "penalized_tanh": penalized_tanh,
    "eliSH": eliSH,
    "mish": mish,
    "rsigelu": rsigelu,
    "tanh_exp": tanh_exp,
    "gcu": gcu
}
for dataset in datasets:
        for activation in activation_functions:
            print(f"🔹 Training on {dataset} with activation: {activation}")
            
            X_train, X_test, y_train, y_test = load_dataset(dataset)
            num_classes = len(np.unique(y_train))
            
            model = create_cnn_model(X_train.shape[1:], activation, num_classes)
            model.summary()
            
            history=model.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_test, y_test))
            
            save_model(model, dataset, activation)
            plot_training(history, dataset, activation)
            
            test_loss, test_acc = model.evaluate(X_test, y_test)
            print(f"📊 Test Accuracy on {dataset} with {activation}: {test_acc:.4f}")
            
            # Save test accuracy results
            save_test_results(dataset, activation, test_loss, test_acc)
            


🔹 Training on fashion_mnist with activation: relu


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 lambda_76 (Lambda)          (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_38 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_39 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 lambda_77 (Lambda)          (None, 14, 14, 64)        0         
                                                                 
 max_pooling2d_39 (MaxPooli  (None, 7, 7, 64)          0         
 ng2D)                                               

2025-03-17 11:06:47.950739: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


938/938 [==============================] - 12s 12ms/step - loss: 0.6190 - accuracy: 0.7996 - val_loss: 0.5280 - val_accuracy: 0.8391
✅ Model weights saved: saved_models/fashion_mnist/cnn_fashion_mnist_relu.h5
✅ Metadata saved: saved_models/fashion_mnist/metadata_fashion_mnist_relu.json
📊 Plot saved as training_graphs/training_fashion_mnist_relu.png
313/313 [==============================] - 2s 5ms/step - loss: 0.5280 - accuracy: 0.8391
📊 Test Accuracy on fashion_mnist with relu: 0.8391
✅ Test accuracy saved in results/fashion_mnist_results.csv
🔹 Training on fashion_mnist with activation: tanh


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 lambda_80 (Lambda)          (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_40 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 lambda_81 (Lambda)          (None, 14, 14, 64)        0         
                                                                 
 max_pooling2d_41 (MaxPooli  (None, 7, 7, 64)          0         
 ng2D)                                               

...

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_92 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 lambda_184 (Lambda)         (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_92 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_93 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 lambda_185 (Lambda)         (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d_93 (MaxPooli  (None, 8, 8, 64)          0         
 ng2D)                                               

2025-03-17 11:17:21.714725: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


782/782 [==============================] - 32s 40ms/step - loss: 3.4982 - accuracy: 0.1860 - val_loss: 2.8507 - val_accuracy: 0.2975
✅ Model weights saved: saved_models/cifar100/cnn_cifar100_rsigelu.h5
✅ Metadata saved: saved_models/cifar100/metadata_cifar100_rsigelu.json
📊 Plot saved as training_graphs/training_cifar100_rsigelu.png
313/313 [==============================] - 4s 13ms/step - loss: 2.8507 - accuracy: 0.2975
📊 Test Accuracy on cifar100 with rsigelu: 0.2975
✅ Test accuracy saved in results/cifar100_results.csv
🔹 Training on cifar100 with activation: tanh_exp


Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_94 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 lambda_188 (Lambda)         (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_94 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_95 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 lambda_189 (Lambda)         (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d_95 (MaxPooli  (None, 8, 8, 64)          0         
 ng2D)                                               

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_96 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 lambda_192 (Lambda)         (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_96 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_97 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 lambda_193 (Lambda)         (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d_97 (MaxPooli  (None, 8, 8, 64)          0         
 ng2D)                                               

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# Define datasets and activation functions
datasets = ["fashion_mnist", "cifar10", "cifar100"]

# Folder for saved graphs
graph_folder = "saved_graphs"
super_graph_folder = "super_graphs"
os.makedirs(super_graph_folder, exist_ok=True)

# Create super graphs for each dataset
for dataset in datasets:
    plt.figure(figsize=(10, 6))

    for activation in activation_functions:
        graph_path = os.path.join(graph_folder, f"training_{dataset}_{activation}.png")
        
        if os.path.exists(graph_path):
            history = np.load(graph_path, allow_pickle=True).item()
            plt.plot(history['accuracy'], label=f"{activation}", color=activation_colors.get(activation, "black"), linewidth=2)
            plt.plot(history["val_accuracy"], label=f"{activation} Val", color=activation_colors.get(activation, "black"), linestyle="--")
    # Graph customization
    plt.title(f"Super Graph - {dataset}", fontsize=14, fontweight="bold")
    plt.xlabel("Epochs", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)
    plt.legend(title="Activation Functions", fontsize=10, title_fontsize="12", loc="best")  # Proper activation labels
    plt.grid(True, linestyle="--", alpha=0.5)  # Light grid for readability
    
    # Save super graph
    super_graph_path = os.path.join(super_graph_folder, f"super_graph_{dataset}.png")
    plt.savefig(super_graph_path, dpi=300)  # High-resolution save
    plt.close()

print(f"✅ Super graphs saved in {super_graph_folder} with clear labels.")


✅ Super graphs saved in super_graphs with clear labels.
